In [1]:
import re
import sys
import json

In [2]:
with open("hida\HidaDaten.uld", "r",encoding="latin-1") as file:
    txt = file.read()
    #txt = txt.replace("\n", " ")
    #txt = txt.replace("„", "ä")
    #txt = txt.replace("”", "ö")
    #txt = txt.replace("á", "ß")
    txt = ' '.join(txt.split())
    
with open('hida\entryKeysHida.json') as h:
  dictEntry = json.load(h)
  singleValue= list(dictEntry['singleValue'])
  del dictEntry['singleValue']

In [3]:
#helper function
def findAllMainObjNr(s):
    pat=r'%s(\d{8}[,]T)' % '5000=\s'
    #pat = r'(5000=\s)' % '\d{8}'
    #pat = r'(\w*%s\w*)' % '\s\d{8}\s'
    list=  re.findall(pat, s)
    return list
#print(findAllMainObjNr(txt))

def findAllSubObjNr(s):
    pat=r'%s(\d{8}.{6})' % '5001=\s'
    list=  re.findall(pat, s)
    return list

def findAllSub2ObjNr(s):
    pat=r'%s(\d{8}.{12})' % '5002=\s'
    list=  re.findall(pat, s)
    return list
#print(findAllSubObjNr(txt))

def findAllObjNr(s):
    pat=r'%s(\d{8}[,]T[,]\d{3}[,]T[,]\d{3}\s|\d{8}[,]T[,]\d{3}[,]T\s|\d{8}[,]T[,]\d{3}\s|\d{8}[,]T\s|\d{8}\s)' % '(5002=\s|5001=\s|5001=\s|5000=\s|5000=\s)'
    listObj=  re.findall(pat, s)
    dict={}
    for obj in listObj:
        dict[obj[1]]=obj[0]
    return dict
#print(findAllObjNr(txt))

def pattern(listPattern):
    listPattern=list(listPattern)
    pat=r'('
    for i in range(len(listPattern)):
        if i!=len(listPattern)-1:
            pat=pat+listPattern[i]+'= |'
        else:
             pat+=listPattern[i]+'= )'
    return pat
#print(pattern(dictEntry.keys()))

#Beziehung e.g. do not explicitely refer to the 09096451,T and 9075895,T  but instead to 09096451 and 9075895
def referBezKeyT(x,y):
    if x not in y:
        x = x+',T'
    return x        

#converts unconverted characters\x88
def replacement(x):
    return x.replace('\x90','[]').replace('\x9b','>').replace('\x9c','oe').replace('\x9d','[]').replace('\x86','+').replace('\x87','[]').replace('\x88','^').replace('\x83','f').replace('\x8a','s').replace('\x8c','OE').replace('\x82','é').replace('\x8e','ä').replace('\x81','ü').replace('\x9a','ü').replace('\x84','ä').replace('\x94','ö').replace('\x99','ö').replace('á','ß').replace('ø','')

In [6]:
def createDict(s):
    objNr=findAllObjNr(s)
    objNrKeys=list(objNr.keys())
    dictObjNrValue={}
    pat=pattern(dictEntry.keys())# search pattern as keys in dictEntry
    for n in range(0,len(objNrKeys)):#14500):#5680
        key=objNrKeys[n]
        val=objNr[key]
        key=key[:-1]#remove empty space
        if n%1000==0:
            my_str = ('Progress : ' +str(round(n/len(objNr.keys())*100,3))+'%\n')
            sys.stdout.write('\r' + str(my_str))
            sys.stdout.flush()
            print('key: ', key, ' value: ', val, ' n: ', n)
        start1 = s.find(val+key)# + len(key+val)
        start2 = 0# 0 since blk is at the 0th position# s.find('blk=')
        #this if else condition is important for the last element since another ObjNr will not follow
        if n !=len(objNr.keys())-1:
            # start1 is required in order to finde the next 'blk=' instead of the current one
            end = s[start1:].find('blk=')+start1
        #for the last element there is no further upcoming objnr
        else: 
            end = len(s)
        value = s[start2:end]
        searcher = re.search(pat, value)
        dictObjNrValue[key]={}
        valueKey=''
        while(bool(searcher)):
            prevKeyCompare=valueKey
            #start save key
            startKey = searcher.span()[0]
            endKey = searcher.span()[1]
            valueKey = value[startKey:endKey].replace('= ', '')
            #end save key
            #find the next key
            searcher = re.search(pat, value[endKey:])
            if bool(searcher):
                startKey2=searcher.span()[0]+endKey
                endKey2 = searcher.span()[1]+endKey
                #end find next key
                #find the content of the first key and put all in the dictionary
                startInhalt= endKey
                endInhalt = startKey2
                setFinalValue=replacement(value[startInhalt:endInhalt-1])#-1 due to extraction of empty space
                value = value[startKey2:]
                searcher = re.search(pat, value)
            else:
                startInhalt= endKey
                setFinalValue=replacement(value[startInhalt:-1])#-1 due to extraction of empty space
            #these keys have only a single value element
            if (dictEntry[valueKey] in singleValue):
                dictObjNrValue[key][dictEntry[valueKey]]=setFinalValue
            #these keys have multiple value elements
            else:
                #if dictEntry[valueKey] as key does not exist create that key with an empty list as value
                dictObjNrValue[key].setdefault(dictEntry[valueKey],[]).append(setFinalValue)
            #save Straße + Hausnummer in one key
            if prevKeyCompare=='5116' and valueKey=='5117':
                strasse=dictObjNrValue[key][dictEntry['5116']][-1]
                hnr=dictObjNrValue[key][dictEntry['5117']][-1].upper()
                strasseHausnummer=strasse+' '+hnr
                dictObjNrValue[key].setdefault('Adresse',[]).append(strasseHausnummer)
                dictObjNrValue[key].setdefault('AdresseDict',{})[strasse]=hnr
            elif prevKeyCompare=='5116' and valueKey!='5117':
                strasse=dictObjNrValue[key][dictEntry['5116']][-1]
                dictObjNrValue[key].setdefault('Adresse',[]).append(strasse)
                dictObjNrValue[key].setdefault('AdresseDict',{})[strasse]=''
            #end
        s=s[end:]
        #here, separate the elements in "Bez-OBJ-Nr" since multiple elements are just separeted by "&". 
        bezObj={}
        if 'Bez-OBJ-Nr' in dictObjNrValue[key].keys():
            bezKeys=dictObjNrValue[key]['Bez-OBJ-Nr']
            bezValue=dictObjNrValue[key]['Beziehung']
            for i in range(len(bezKeys)):
                if '&' in bezKeys[i]:
                    find = re.findall(r'\d{8}', bezKeys[i])
                    for elBezKey in find:
                        bezObj[elBezKey]=bezValue[i]
                else:
                    bezObj[bezKeys[i]]=bezValue[i]
            del dictObjNrValue[key]['Bez-OBJ-Nr']
            dictObjNrValue[key]['Beziehung']=bezObj
        #end separation
        #here, separate the elements in "Sachbegriff" since multiple elements are just separeted by "&". 
        if 'Sachbegriff' in dictObjNrValue[key].keys():
            sBegriff=dictObjNrValue[key]['Sachbegriff']
            if '&' in sBegriff:
                find = sBegriff.split('&')
            else:
                find=[sBegriff]
            dictObjNrValue[key]['Sachbegriff']=find
        #end separation
        #here, separate the elements in "Denkmalname" since multiple elements are just separeted by "&". 
        if 'Denkmalname' in dictObjNrValue[key].keys():
            dName=dictObjNrValue[key]['Denkmalname']
            if '&' in dName:
                find = dName.split('&')
            else:
                find=[dName]
            dictObjNrValue[key]['Denkmalname']=find
        #end separation
        #here, separate the elements in "Hausnummer" and in Strasse since multiple elements of them are just separeted by "&". 
        adrObj={}
        if 'AdresseDict' in dictObjNrValue[key].keys():
            adrKeys=list(dictObjNrValue[key]['AdresseDict'].keys())#Strasse
            adrValue=list(dictObjNrValue[key]['AdresseDict'].values())#Hausnummer
            for i in range(len(adrValue)):
                if '&' in adrValue[i]:
                    find = re.findall(r'\d+\w*', adrValue[i])
                    adrObj[adrKeys[i]]=find
                elif adrValue[i]=='':
                    adrObj[adrKeys[i]]=[]
                else:
                    adrObj[adrKeys[i]]=[adrValue[i]]
            adrObjKeys=list(adrObj.keys())
            adrObjValue=list(adrObj.values())
            for i in range(len(adrObjKeys)):
                if '&' in adrObjKeys[i]:
                    find = adrObjKeys[i].split('&')
                    del adrObj[adrKeys[i]]
                    for elFind in find:
                        elFind=' '.join(elFind.split())#remove unneccessary spaces
                        adrObj[elFind]=adrObjValue[i]
            dictObjNrValue[key]['AdresseDict']=adrObj
        #end separation
        includeList=['Sachbegriff','Denkmalname']
        for include in includeList:
            if include not in dictObjNrValue[key].keys():
                dictObjNrValue[key][include]=[]
            
    #print(dictObjNrValue)
    return dictObjNrValue

In [7]:
#save objNrValue as json file
with open('HidaDict.json', 'w') as fp:
    json.dump(createDict(txt), fp,  indent=4,ensure_ascii=False)
print('successfully saved')

Progress : 0.0%
key:  09097892  value:  5000=   n:  0
Progress : 6.875%
key:  09095735,T,002  value:  5001=   n:  1000
Progress : 13.749%
key:  09085800  value:  5000=   n:  2000
Progress : 20.624%
key:  09080565  value:  5000=   n:  3000
Progress : 27.499%
key:  09075517,T,002  value:  5001=   n:  4000
Progress : 34.374%
key:  09066564  value:  5000=   n:  5000
Progress : 41.248%
key:  09065666  value:  5000=   n:  6000
Progress : 48.123%
key:  09050515  value:  5000=   n:  7000
Progress : 54.998%
key:  09046317,T,002  value:  5001=   n:  8000
Progress : 61.873%
key:  09045491  value:  5000=   n:  9000
Progress : 68.747%
key:  09040221  value:  5000=   n:  10000
Progress : 75.622%
key:  09030867  value:  5000=   n:  11000
Progress : 82.497%
key:  09020849  value:  5000=   n:  12000
Progress : 89.372%
key:  09012397,T,002  value:  5001=   n:  13000
Progress : 96.246%
key:  09011503  value:  5000=   n:  14000
successfully saved
